<a href="https://colab.research.google.com/github/kwangwonland/kfq_ML_2023/blob/main/0726_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/acllmdb_v1.tar.gz

--2023-07-26 00:09:17--  https://ai.stanford.edu/~amaas/data/sentiment/acllmdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-07-26 00:09:18 ERROR 404: Not Found.



In [2]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2023-07-26 00:12:02--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  21.4MB/s    in 5.9s    

2023-07-26 00:12:08 (13.5 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [4]:
!tar -xf aclImdb_v1.tar.gz

In [5]:
import tensorflow as tf
from tensorflow import keras

In [8]:
dataset=keras.utils.text_dataset_from_directory(
    directory='aclImdb',
    label_mode=None,
    batch_size=256
)
dataset=dataset.map(lambda x:tf.strings.regex_replace(x,'<br /',' '))

Found 100006 files belonging to 1 classes.


In [9]:
for batch in dataset:
    print(len(batch))
    for text in batch:
        print(text)
        break
    break

256
tf.Tensor(b"This film NAILS it by juxtaposing the fast rise and rocky descent of 80's counter-culture commercialism with the tragic story of Gator and his victim. We get to know Gator as a wild punk, arrogant jock, immature romantic, and finally, a violent and dangerous man. He was made for stardom, but the path to skater stardom was even younger than its pioneers... he was the era's most notable fallen angel.  > >I haven't yet seen a more vivid reflection on the style and attitude of California in the 1980s. It's very easy to get lost in this one; it's a sad, enlightening, and socially significant piece of journalistic film-making. Kudos to the filmmakers.", shape=(), dtype=string)


In [10]:
from keras import layers

In [12]:
sequence_length=100
vocab_size=15000
tv=layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)
tv.adapt(dataset)

In [14]:
for batch in dataset:
    print(len(batch))
    for text in batch:
        print(text)
        vs=tv(text)
        print(vs)
        break
    break

256
tf.Tensor(b'This movie was terrible. It is not something that people should have to pay to see. It looks like some Christian group made it to convert people. I don\'t understand why it was released to theaters and not to TV. > >It started out like an old fashioned B movie sci-fi film from the 50s, but quickly changed. About 30 minutes into the movie the characters start talking about "God" and "Do you believe in Jesus?" It quickly moves into pure religion territory. > >I thought I was going to a Sci-Fi movie. The film has poor acting; bad camera angles and is amateurish. > >AVOID IT!', shape=(), dtype=string)
tf.Tensor(
[   11    17    13   379     9     7    22   137    12    83   140    25
     6   975     6    65     9   279    38    46  1630   516    96     9
     6 11901    83    10    89   375   132     9    13   632     6  2090
     3    22     6   245     9   655    45    38    33   171  4875   962
    17   866    19    35     2  2135    18   912  1153    42  1193   227
   

In [16]:
for batch in dataset:
  vs = tv(batch)
  x = vs[:,:-1]
  y = vs[:,1:]
  break
print(vs[0])
print(x[0])
print(y[0])

tf.Tensor(
[    9   181     2  1078     5    11    19   195    34   910     6  6551
     2  3657     8   618     6    95     4    64    12     7     1     3
 14974  3710  9488    40     4  2323    34   398    25  2361     4    64
    12    13   156   293     3     8     2  3657  1718  3140  5467     5
     1    20  5693     3 11859    16  3145   878  8216   317   325    37
    13  2475   317   488    39   137     8   196    36   675     3    36
  2028     2  3657   110   696   170     3    54   409  1264  5350  1636
    85  3943    54  3568    66   121   225     5  2475   350    42  9601
   151   101    24   315], shape=(100,), dtype=int64)
tf.Tensor(
[    9   181     2  1078     5    11    19   195    34   910     6  6551
     2  3657     8   618     6    95     4    64    12     7     1     3
 14974  3710  9488    40     4  2323    34   398    25  2361     4    64
    12    13   156   293     3     8     2  3657  1718  3140  5467     5
     1    20  5693     3 11859    16  3145   878

In [19]:
def prepare_Im_dataset(text_batch):
    vs=tv(text_batch)
    x=vs[:,:-1]
    y=vs[:1,1:]
    return x,y

In [20]:
Im_dataset=dataset.map(prepare_Im_dataset,num_parallel_calls=4)

In [24]:
embed_dim = 256
latent_dim = 2048
num_heads = 2

In [27]:
from tensorflow.keras import layers

In [39]:
class PositionalEmbedding(layers.Layer):
  def __init__(self,sequence_length,input_dim,output_dim,**kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(input_dim,output_dim)
    self.position_embeddings = layers.Embedding(sequence_length,output_dim)
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim
  def call(self,inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0,limit=length,delta=1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions
  def compute_mask(self,inputs,mask=None):
    return tf.math.not_equal(inputs,0)
  def get_config(self):
    config = super(PositionalEmbedding,self).get_config()
    config.update(
        {
            "output_dim":self.output_dim,
            "sequence_length":self.sequence_length,
            "input_dim":self.input_dim,
        }
    )
    return config

In [40]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [41]:
inputs = keras.Input(shape=(None,),dtype='int64')
x = PositionalEmbedding(sequence_length,vocab_size,embed_dim)(inputs)
x = TransformerDecoder(embed_dim,latent_dim,num_heads)(x,x)
outputs = layers.Dense(vocab_size,activation='softmax')(x)
model = keras.Model(inputs,outputs)
model.compile(loss = keras.losses.sparse_categorical_crossentropy)

In [43]:
import numpy as np
tokens_index=dict(enumerate(tv.get_vocabulary()))

In [44]:
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

In [45]:
class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            sentence = self.prompt
            for i in range(self.generate_length):
                tokenized_sentence = tv([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(predictions[0, i, :])
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token
            print(sentence)

In [49]:
prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

In [50]:
model.fit(Im_dataset,epochs=3,callbacks=[text_gen_callback])

Epoch 1/3


InvalidArgumentError: ignored